In [3]:
import contextlib
import os
import re
import sys

import linearmodels as lm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns
import tabulate

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.figures.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = hd.read_sample("XX7")
hd.inspect(df)

Time for read_sample    : 10.69 minutes
(16,094,005, 31)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac
0,1088993,2012-04-02,57,23.74,1982 01apr 12 the broadway fruiterer london gb,NaN,NaN,NaN,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-10-23,True,NaN,NaN,NaN,NaN,NaN,u,201204,NaN,22542.600586,2.425209
1,1088994,2012-04-02,57,25.68,1982 01apr 12 waitrose 835 crouch end gb,waitrose,spend,household,0.0,n8 9,2010-05-13,10k to 20k,1987.0,2016-10-30,273998,2015-08-15 21:37:00,natwest bank,current,2014-07-18,2017-08-15,True,NaN,waitrose,"food, groceries, household",NaN,supermarket,u,201204,NaN,22542.600586,2.425209


Min monthly obs

In [5]:
vl.no_gaps_and_min_obs(df)

AssertionError: 

In [17]:
dd = df.set_index(["user_id", "date"])

Identify users

In [89]:
a = dd.groupby("user_id").resample("m", level="date").size().groupby("user_id").min()

In [120]:
users = a[a < 10].index
users

Int64Index([225137, 274057, 282357, 419397, 438657, 441977, 448837, 458627,
            554947],
           dtype='int64', name='user_id')

In [121]:
b = dd.groupby(["user_id", "ym"]).size().groupby("user_id").min()
users2 = b[b < 10].index

assert all(users2 == users)
len(users)

9

Finding error

In [114]:
def min_spend(df, min_txns=10, min_spend=200):
    """At least 5 debits totalling \pounds200 per month

    A minimal test to ensure that we observe a user's full financial life.

    Doesn't account for missing data in first and last months since these
    months were removed for all users during cleaning.
    """
    data = df.loc[df.debit, ["user_id", "id", "ym", "amount"]]
    g = data.groupby(["user_id", "ym"])
    min_monthly_spend = g.amount.sum().groupby("user_id").min()
    min_monthly_txns = g.size().groupby("user_id").min()
    conds = min_monthly_spend.ge(min_spend) & min_monthly_txns.ge(min_txns)
    users = conds[conds].index
    return df.loc[df.user_id.isin(users)]


def min_spend(df, min_debits=10, min_spend=200):
    """At least 10 debits totalling \pounds200 per month

    A minimal test to ensure that we observe a user's full financial life.

    Doesn't account for missing data in first and last months since these
    months were removed for all users during cleaning.
    """
    d = df.loc[:, ["user_id", "ym", "debit"]]
    min_monthly_debits = (
        d.groupby(["user_id", "ym"]).debit.sum().groupby("user_id").min()
    )

    d = df.loc[df.debit, ["user_id", "ym", "amount"]]
    min_monthly_spend = (
        d.groupby(["user_id", "ym"]).amount.sum().groupby("user_id").min()
    )

    conds = min_monthly_spend.ge(min_spend) & min_monthly_debits.ge(min_debits)
    users = conds[conds].index
    return df.loc[df.user_id.isin(users)]


min_spend(df).user_id.nunique()

5677

In [62]:
df.user_id.nunique()

5686